In [2]:
import new_adventure as na
import new_adventure.derivative_free_estimation as dfe
import numpy as np
import matplotlib.pyplot as plt
import time

import numpy as np
import time

import multiprocessing
from os import getpid

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
def new_beta_second_shift_estimator(F, x_0, alpha, N, control_variate=True, num_processes=1):

    def process_task(process_N, L_sample_points, L_grads, pid):
        # Notice, we should be sharing the radius. So some lock is probably needed to synchronise 
        sample_points = dfe.hit_run(x_0, F, x_0.shape[0], process_N, alpha)
        out_grads = F.f1(sample_points)
        L_sample_points.append(sample_points)
        L_grads.append(out_grads)

    with multiprocessing.Manager() as manager:
        L_sample_points = manager.list()
        L_grads = manager.list()
        for i in range(num_processes):
            p = multiprocessing.Process(target=process_task, args=(N // num_processes, L_sample_points, L_grads, i))
            p.start()
            p.join()

        sample_points = np.vstack(list(L_sample_points))
        out_grads = np.vstack(list(L_grads))

    second_shift_est = dfe.new_proper_cov(sample_points, out_grads)
    return second_shift_est.dot(np.linalg.inv(dfe.np_new_cov(sample_points)))

In [54]:
np.random.seed(10)
dim = 5000
num_barriers = dim * 2
dirs = np.random.normal(size=(num_barriers, dim)) # sample gaussian and normalize 
ws = dirs/np.linalg.norm(dirs, axis=1).reshape(-1, 1)
bs = np.ones(num_barriers)

barrier = na.Barriers.LogPolytopeBarrier(ws, bs)

F = na.Functions.Linear(np.ones(dim))

F = barrier # na.Functions.LinearCombination(F, barrier, [1, 1])

In [ ]:
xs = np.ones(dim) / np.linalg.norm(np.ones(dim))

a = time.time()
new_beta_second_shift_estimator(F, xs, 200, 2500, control_variate=True)
print(time.time() - a)

a = time.time()
new_beta_second_shift_estimator(F, xs, 200, 2500, control_variate=True, num_processes=10)
print(time.time() - a)

62.857404947280884


In [52]:
dfe.new_beta_second_shift_estimator(F, xs, 200, 2500, control_variate=True)

array([[ 2.04657718,  0.06195903,  0.01523212, ..., -0.04717341,
         0.07388959, -0.13664766],
       [ 0.06170182,  2.03244957, -0.10863438, ..., -0.01839577,
         0.10105179, -0.19522663],
       [ 0.01925689, -0.10880495,  2.07177371, ..., -0.04647866,
        -0.05948505, -0.06567129],
       ...,
       [-0.04851404, -0.01977485, -0.04509338, ...,  1.96665647,
         0.01855206, -0.01992453],
       [ 0.07666633,  0.09941025, -0.05855921, ...,  0.01526542,
         1.93963773,  0.01887988],
       [-0.13735084, -0.19386065, -0.0634109 , ..., -0.02254701,
         0.0241202 ,  1.9790937 ]])